In [2]:
import numpy as np
import torch
from PIL import Image
import torchvision.transforms.functional as TF
import torchvision.transforms as transforms

In [3]:
def predict_real_fake(
    face_image: np.ndarray, classifier_model: torch.nn.Module, device: torch.device
):
    image = Image.fromarray(face_image)

    transform = transforms.Compose(
        [
            transforms.Resize(400),  # Resize to a larger size first (zoom in)
            transforms.CenterCrop(299),  # Then crop the center to 299x299
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ]
    )

    input_tensor = transform(image).unsqueeze(0)
    input_tensor = input_tensor.to(device)

    classifier_model.eval()
    classifier_model.to(device)
    with torch.no_grad():
        outputs = classifier_model(input_tensor)
        probabilities = torch.softmax(outputs, dim=1)
        predicted_class = torch.argmax(probabilities, dim=1).item()

    return predicted_class, probabilities.cpu().numpy()

In [4]:
from dataset.facenet import create_model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
classifier_model = create_model(num_classes=2)
classifier_model.load_state_dict(torch.load("model/real_fake_model.pth"))
classifier_model.to(device)

C:\Users\trung\AppData\Local\Temp\ipykernel_13592\544197719.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  classifier_model.load_state_dict(torch.load("model/real_fake_

FaceNet(
  (Conv2d_1a): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
  )
  (Conv2d_2a): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
  )
  (Conv2d_2b): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (Conv2d_3b): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): Re

In [ ]:
image_path = "dataset/real/Screenshot 2025-03-11 134133.jpg"
face_image = np.array(Image.open(image_path))
print(face_image)
# Make prediction
predicted_class, probabilities = predict_real_fake(face_image, classifier_model, device)

# Output the prediction
print(f"Predicted class: {predicted_class} (0 = Real, 1 = Fake)")
print(f"Probabilities: {probabilities}")

[[[  0   6 236]
  [  0   6 242]
  [  0   3 252]
  ...
  [  0   0 237]
  [  6   6 228]
  [ 72  72 255]]

 [[  0   8 245]
  [  0   6 244]
  [  0   0 237]
  ...
  [  0   0 240]
  [  5   5 227]
  [ 72  72 255]]

 [[  0   0 248]
  [  0   0 235]
  [ 13   6 224]
  ...
  [  3   2 243]
  [  4   4 226]
  [ 72  72 255]]

 ...

 [[  0   0 252]
  [  1   0 251]
  [  0   1 244]
  ...
  [  0   0 242]
  [  0   6 232]
  [ 13  28 243]]

 [[  8   5 244]
  [  0   0 237]
  [ 10  10 255]
  ...
  [  0   2 238]
  [  0  13 226]
  [ 18  37 237]]

 [[ 77  86 255]
  [  8  18 229]
  [  0   3 235]
  ...
  [  0   7 225]
  [ 14  36 220]
  [ 46  73 240]]]
Predicted class: 0 (0 = Real, 1 = Fake)
Probabilities: [[0.99352926 0.00647077]]
